In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dependencies

In [28]:
import datasets_handler
import evaluation_metrics
import zeroberto
from zeroberto import ZeroBERTo
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import gc

In [29]:
which_dataset = 'folhauol' 
max_inferences = 20000

hyp_template = "Esse texto está relacionado a {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())

zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual',
    'setfit_model' : 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2',# 'sentence-transformers/stsb-xlm-r-multilingual',
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"zeroshot",
    'method':zeroshot_method,
    'max_inferences':max_inferences,
    'classes':classes_list,
    'template': hyp_template,
    'random_state':422,
    'top_n': 8,
    'n_examples': 8,
    "batch_size" : 4,
    "num_pairs" : 20,
    "num_epochs" : 1,
    'st_train_epochs': 10,
    'st_train_batch_size': 20,
    # Unfreeze the head and freeze the body -> head-only training
    'keep_body_frozen_setfit': True,
    # Unfreeze the head and unfreeze the body -> end-to-end training
    # zeroshot_config['keep_body_frozen_setfit'] = False
}

data_to_label = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()

model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
                  
model.runLabeling()
model.evaluateLabeling()

model.getLabelingMetrics()
model.saveLabelingResults()


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Preds: 100  - Total time: 8.83 seconds - ETA: 29.4 minutes
Preds: 200  - Total time: 16.37 seconds - ETA: 27.3 minutes
Preds: 300  - Total time: 24.63 seconds - ETA: 27.4 minutes
Preds: 400  - Total time: 32.85 seconds - ETA: 27.4 minutes
Preds: 500  - Total time: 40.55 seconds - ETA: 27.0 minutes
Preds: 600  - Total time: 47.64 seconds - ETA: 26.5 minutes
Preds: 700  - Total time: 54.74 seconds - ETA: 26.1 minutes
Preds: 800  - Total time: 61.89 seconds - ETA: 25.8 minutes
Preds: 900  - Total time: 69.19 seconds - ETA: 25.6 minutes
Preds: 1000  - Total time: 78.1 seconds - ETA: 26.0 minutes
Preds: 1100  - Total time: 86.19 seconds - ETA: 26.1 minutes
Preds: 1200  - Total time: 94.18 seconds - ETA: 26.2 minutes
Preds: 1300  - Total time: 101.93 seconds - ETA: 26.1 minutes
Preds: 1400  - Total time: 109.44 seconds - ETA: 26.1 minutes
Preds: 1500  - Total time: 117.02 seconds - ETA: 26.0 minutes
Preds: 1600  - Total time: 125.3 seconds - ETA: 26.1 minutes
Preds: 1700  - Total time: 132.4

In [36]:
### load results from labeling step and create training data for contrastive learning
zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(
    model.config['dataset'],model.config['class_col'],top_n=model.config['top_n'],exec_time=model.config['exec_time'])

raw_data_final, model.config['new_class_col'] = datasets_handler.mergeLabelingToDataset(
    raw_data,zeroshot_previous_data,model.config['class_col'])

df_train, df_test = datasets_handler.splitDataset(raw_data_final,model.config) ### df_test will not be used
train_dataset = datasets_handler.buildDatasetDict(df_train)

1666


In [37]:
### overrides config for looping second part of the model
# model.config['num_epochs'] = 5
# model.config['num_pairs'] = 20
# model.config['keep_body_frozen_setfit'] = False

### build trainer for contrastive learning
model.buildTrainer(train_dataset)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
model.getPredictions()
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese

setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, model.trainer.eval_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics)
print(model.config)

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 128.


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running predictions on 20000 sentences.
